In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder as le
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LogisticRegression as lr
from sklearn.metrics import mean_absolute_error

In [4]:
df = pd.read_csv("../data/mental_wellness.csv")
mentalHealthGroups = ["bad_mental_health","average_mental_health","good_mental_health"]
mentalHealthBounderies = [0,33,66,100]
stressLevelGroups = ["Low_stress","manageble_stress","High_stress"]
stressLevelBounderies = [0,3,6,10]
productivityGroups = ["no_productivity","satisfiable_productivity","peak_productivity"]
productivityBounderies = [0,33,66,100]
df["mental_wellness_grouped"] = pd.cut(df["mental_wellness_index_0_100"],bins=mentalHealthBounderies,labels=mentalHealthGroups,include_lowest=True)
df["stress_level_grouped"] = pd.cut(df["stress_level_0_10"],bins=stressLevelBounderies,labels=stressLevelGroups,include_lowest=True)
df["productivity_grouped"] = pd.cut(df["productivity_0_100"],bins=productivityBounderies,labels=productivityGroups,include_lowest=True)
df = df.drop(["mental_wellness_index_0_100","stress_level_0_10","productivity_0_100","Unnamed: 15"],axis=1)
df.head(2)

,user_id,age,gender,occupation,work_mode,screen_time_hours,work_screen_hours,leisure_screen_hours,sleep_hours,sleep_quality_1_5,exercise_minutes_per_week,social_hours_per_week,mental_wellness_grouped,stress_level_grouped,productivity_grouped
0,U0001,33,Female,Employed,Remote,10.79,5.44,5.35,6.63,1,127,0.7,bad_mental_health,High_stress,satisfiable_productivity
1,U0002,28,Female,Employed,In-person,7.40,0.37,7.03,8.05,3,74,2.1,average_mental_health,manageble_stress,peak_productivity


In [5]:
encoder = le()
df["gender_encoded"] = encoder.fit_transform(df["gender"])
df["occupation_encoded"] = encoder.fit_transform(df["occupation"])
df["work_mode_encoded"] = encoder.fit_transform(df["work_mode"])
df["stress_level_encoded"] = encoder.fit_transform(df["stress_level_grouped"])
df["productivity_encoded"] = encoder.fit_transform(df["productivity_grouped"])
df["mental_wellness_grouped_encoded"] = encoder.fit_transform(df["mental_wellness_grouped"])
df = df.drop(["gender","occupation","work_mode","user_id","mental_wellness_grouped","stress_level_grouped","productivity_grouped"],axis=1)
df.head(2)

,age,screen_time_hours,work_screen_hours,leisure_screen_hours,sleep_hours,sleep_quality_1_5,exercise_minutes_per_week,social_hours_per_week,gender_encoded,occupation_encoded,work_mode_encoded,stress_level_encoded,productivity_encoded,mental_wellness_grouped_encoded
0,33,10.79,5.44,5.35,6.63,1,127,0.7,0,0,2,0,2,1
1,28,7.40,0.37,7.03,8.05,3,74,2.1,0,0,1,2,1,0


In [6]:
X = df.drop("mental_wellness_grouped_encoded",axis=1)
Y = df["mental_wellness_grouped_encoded"]
X_train,X_test,Y_train,Y_test = tts(X,Y,test_size=0.2,random_state=42)
print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)

(320, 13) (320,) (80, 13) (80,)


In [7]:
logisticRegressionModel = lr()
logisticRegressionModel.fit(X_train,Y_train)

E:\data_ai\final project\finalProject\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [8]:
Y_pred = logisticRegressionModel.predict(X_test)
print(Y_pred[:5],Y_test[:5])

[1 1 1 1 1] 209    1
280    1
33     1
210    1
93     1
Name: mental_wellness_grouped_encoded, dtype: int64


In [9]:
print(logisticRegressionModel.score(X_test,Y_test))
print(mean_absolute_error(Y_test,Y_pred))

0.875
0.175


In [17]:
import numpy as np
Y_test = np.array(Y_test)
count=0
for i in range(len(Y_pred)):
    if(Y_pred[i]==Y_test[i]):
        count=count+1
print("exactly right values as a precentage "+str(count/len(Y_pred)*100)+"%")

right values as a precentage 87.5%
